# Problem-Specific Coordinate Generation for HyperNEAT Substrates

## Setup

The testing was done on a 16GB VRAM GPU with CUDA 12.8. VRAM usage is determined mainly by substrate and population size.

### Imports

This setup requires some dependencies, mainly TensorNEAT, JAX, numpy, matplotlib, NetworkX, scikit-learn and wandb for logging. Using a virtual environment (i.e. conda) is highly recommended. Python 3.10.18 was used in development and testing.

In [1]:
import wandb
import pickle

from config import config
from evol_pipeline.custom_pipeline import CustomPipeline
from utils.visualization import visualize_cppn, visualize_nn
from utils.utils import setup_folder_structure
from evol_pipeline.evol_algorithm import create_evol_algorithm
from evol_pipeline.brax_env import CustomBraxEnv

A quick setup of the folder structure to avoid errors further down the line.

In [2]:
OUTPUT_DIR = config["experiment"]["output_dir"]
setup_folder_structure(OUTPUT_DIR)

with open(f"{OUTPUT_DIR}/substrates.pkl", "rb") as f:
    substrates = pickle.load(f)

env_name = config["experiment"]["env_name"]
env_problem = CustomBraxEnv(
    env_name=env_name,
    backend=config["environment"]["backend"],
    brax_args=config["environment"]["brax_args"],
    max_step=config["environment"]["max_step"],
    repeat_times=config["environment"]["repeat_times"],
    obs_normalization=False,
    sample_episodes=16,
)
obs_size = env_problem.input_shape[0]
act_size = env_problem.output_shape[0]

FileNotFoundError: [Errno 2] No such file or directory: 'output/ant/substrates.pkl'

## Neuroevolution

Finally the substrates are ready to be used for neuroevolution.


In [ ]:
for data_label, data_dict in substrates.items():
    for method_label, method_dict in data_dict.items():

        active_substrate = substrates[data_label][method_label]["substrate"]
        evol_algorithm = create_evol_algorithm(substrate=active_substrate)

        wanbd_name = f"{env_name}_{data_label}_{method_label}"
        wandb_tags = [config["substrate"]["hidden_layer_type"], env_name, data_label, method_label]

        wandb.init(
            name=wanbd_name,
            project="substrate_dims",
            tags=wandb_tags,
            config=config  
        )

        wandb.config.update({
            "substrate": {
                "obs_size": obs_size,
                "act_size": act_size,
                "substrate.query_coors.shape": active_substrate.query_coors.shape, # (num_queries, query_dim)
                "algorithm.num_inputs": evol_algorithm.num_inputs,
                }
            }
        )

        pipeline = CustomPipeline(
            algorithm=evol_algorithm,
            problem=env_problem,
            seed=config["experiment"]["seed"],
            generation_limit=config["evolution"]["generation_limit"],
            fitness_target=config["evolution"]["fitness_target"],
            is_save=False,
            save_dir=config["experiment"]["output_dir"],
        )

        init_state = pipeline.setup()
        state = pipeline.auto_run(state=init_state)

        print(f"\nTraining finished. Best fitness achieved: {pipeline.best_fitness}")

        wandb.finish()

        state_for_show = state[0] if isinstance(state, tuple) else state

        # Transform the best genome into network parameters
        best_genome = pipeline.best_genome

        # Built-in show method to produce and save video
        pipeline.show(
            state=state_for_show,
            best=best_genome,
            output_type="mp4",
            save_path=f"{OUTPUT_DIR}/video/agent_{data_label}_{method_label}.mp4",
        )

        input_coors = substrates[data_label][method_label]["input_coors"]
        hidden_coors = substrates[data_label][method_label]["hidden_coors"]
        output_coors = substrates[data_label][method_label]["output_coors"]
        visualize_cppn(pipeline, state, f"{OUTPUT_DIR}/topology/cppn_{data_label}_{method_label}.svg")
        visualize_nn(pipeline, state, f"{OUTPUT_DIR}/topology/nn_{data_label}_{method_label}.svg", active_substrate, input_coors, hidden_coors, output_coors, config["substrate"]["hidden_depth"])


wandb: Currently logged in as: wirkelzirkel (wirkelzirkel-iu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


initializing
initializing finished
start compile
compile finished, cost time: 22.812298s
Generation: 1, Cost time: 8801.63ms
 	fitness: valid cnt: 600, max: 22.7817, min: -171.5821, mean: -50.7192, std: 43.2601

	node counts: max: 7, min: 5, mean: 6.09
 	conn counts: max: 7, min: 0, mean: 4.70
 	species: 15, [260, 3, 90, 4, 1, 2, 1, 38, 1, 63, 2, 1, 1, 13, 120]

Generation: 2, Cost time: 8869.04ms
 	fitness: valid cnt: 600, max: 122.0548, min: -313.4383, mean: -24.5565, std: 40.8689

	node counts: max: 8, min: 5, mean: 6.09
 	conn counts: max: 8, min: 0, mean: 4.17
 	species: 15, [93, 49, 117, 48, 77, 24, 16, 23, 14, 3, 9, 5, 5, 2, 115]

Generation: 3, Cost time: 8846.00ms
 	fitness: valid cnt: 600, max: 132.6757, min: -193.0689, mean: -15.7432, std: 35.8492

	node counts: max: 8, min: 5, mean: 6.17
 	conn counts: max: 8, min: 0, mean: 4.17
 	species: 15, [53, 72, 34, 61, 39, 28, 24, 76, 21, 16, 24, 14, 12, 4, 122]

Generation: 4, Cost time: 8836.82ms
 	fitness: valid cnt: 600, max: 13